In [ ]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYjNiYmZhYjEtNzc3ZS00Y2NhLWI5NTgtYWU0MmQyMWJhM2I0In0="
os.environ['NEPTUNE_PROJECT']="jgeof/sdae"
os.environ['NEPTUNE_NOTEBOOK_ID']="ecd86e96-4da7-44e3-9a17-43da2dfcae35"
os.environ['NEPTUNE_NOTEBOOK_PATH']="constrained-SDAE/sdae.ipynb"
import pandas as pd
import numpy as np
from datetime import datetime
from models import Autoencoder, EncoderStack
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import normalize
import dataset
import neptune

In [ ]:
!neptune tensorboard ./log_dir --project ${NEPTUNE_PROJECT}

In [ ]:
!tensorboard --logdir ./log_dir --bind_all

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.2.2 at http://localhost:6006/ (Press CTRL+C to quit)


In [2]:
config = {
    "N_NODES": [2000, 1000, 500],
    "DROPOUT": [0.1],
    "BATCH_SIZE": 3,
    "EPOCHS": 10,
    "TEST_RATIO": 0.15,
    "DATA_BUCKET": "sdae-geo",
    "DATA_OBJECT": "GEO_data_batch_corr_final.csv",
    "DATA_LABELS": " GBM_class.csv",
    "VERBOSITY": 1
}


In [3]:
neptune.init(os.environ['NEPTUNE_PROJECT'], api_token=os.environ['NEPTUNE_API_TOKEN'])
neptune.create_experiment(name='configuration', params=config)
neptune.append_tag('minimal-example')

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/jgeof/sdae/e/SDAE-12


# Load and preprocess the data

## Load data from Google Storage

In [4]:
dataframe = dataset.load_gs_data(config['DATA_BUCKET'], config['DATA_OBJECT'])

Initializing dataset...
Blob GEO_data_batch_corr_final.csv downloaded to ./data/GEO_data_batch_corr_final.csv.
Loaded 954 samples with 6785 features.


## Normalize data feature-wize

In [5]:
data = dataframe.values
data = normalize(data)

## Split into training and testing sets

In [6]:
rs = ShuffleSplit(n_splits=1, test_size=config['TEST_RATIO'], random_state=0)
split_itterator = rs.split(data)
i_train, i_test = next(split_itterator)
np.save("train_indices.npy", i_train)
np.save("test_indices.npy", i_test)
neptune.log_artifact('train_indices.npy')
neptune.log_artifact('test_indices.npy')

x_train, x_test = data[i_train], data[i_test]

print("{}% of samples for training: {} training, {} testing.".format(int(config['TEST_RATIO']*100), len(i_train), len(i_test)))

15% of samples for training: 810 training, 144 testing.


# Train encoder layers

In [ ]:
encoder_models = []

x_train, x_test = data, data
for idx, num_hidden in enumerate(config["N_NODES"]):
    print("Training layer {} with {} hidden nodes..\n".format(idx, num_hidden))
    
    encoder = Autoencoder(x_train.shape[1], num_hidden, "output/")
    
    recon_mse = encoder.fit(x_train, x_test, batch_size=config["BATCH_SIZE"], 
        num_epochs=config["EPOCHS"], verbose=config["VERBOSITY"])
    
    x_train_out = encoder.encoder_model.predict(x_train_out)
    x_test_out = encoder.encoder_model.predict(x_test_out)
    print("\nTraining losss: ", recon_mse[0])
    print("\nTesting loss: ", recon_mse[1])
    encoder_models.append(encoder)

Training layer 0 with 2000 hidden nodes..

Epoch 1/10
318/318 [==============================] - 64s 202ms/step - loss: 0.0320 - val_loss: 1.9084e-04
Epoch 2/10
318/318 [==============================] - 67s 212ms/step - loss: 6.1792e-05 - val_loss: 2.1431e-05
Epoch 3/10
318/318 [==============================] - 63s 199ms/step - loss: 1.5967e-05 - val_loss: 9.7162e-06
Epoch 4/10
318/318 [==============================] - 64s 201ms/step - loss: 9.1267e-06 - val_loss: 5.9471e-06
Epoch 5/10
 13/318 [>.............................] - ETA: 55s - loss: 7.1620e-06

# Train encoder stack

In [11]:
model = EncoderStack(encoder_models, 'output/')

print("\n##################################################################")
print("Training layer {} with {} hidden nodes..\n".format(idx, num_hidden))
loss_train, loss_test = model.fit(x_train, y_train, x_test, y_test, batch_size=BATCH_SIZE, num_epochs=EPOCHS)

print("\nTraining losss: ", loss_train)
print("\nTesting loss: ", loss_test)


##################################################################
Training layer 2 with 500 hidden nodes..

Epoch 1/10
270/270 [==============================] - 46s 172ms/step - loss: 0.0912 - val_loss: 0.0657
Epoch 2/10
270/270 [==============================] - 50s 184ms/step - loss: 0.0893 - val_loss: 0.0655
Epoch 3/10
270/270 [==============================] - 53s 195ms/step - loss: 0.0844 - val_loss: 0.0565
Epoch 4/10
270/270 [==============================] - 50s 184ms/step - loss: 0.0685 - val_loss: 0.0418
Epoch 5/10
270/270 [==============================] - 49s 183ms/step - loss: 0.0541 - val_loss: 0.0389
Epoch 6/10
270/270 [==============================] - 46s 170ms/step - loss: 0.0467 - val_loss: 0.0351
Epoch 7/10
270/270 [==============================] - 47s 173ms/step - loss: 0.0436 - val_loss: 0.0416
Epoch 00007: early stopping
INFO:tensorflow:Assets written to: output/20200727-053055/encoder_stack/assets

Training losss:  0.27949008

Testing loss:  0.2811888
